In [ ]:
! pip install pytorch-lightning~=1.5.0
! pip install datasets~=2.1.0
! pip install transformers
! pip install torchmetrics~=0.7.0
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import load_dataset
from datasets import load_metric
import torch
import torch.nn as nn
from transformers import RobertaTokenizerFast,RobertaForSequenceClassification, AdamW, TrainingArguments, Trainer,AutoModelForSequenceClassification
dataset = load_dataset("McGill-NLP/FaithDial")
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)
itemsUnique=[]
for items in dataset["train"]["VRM"]:
    for val in items:
        if val not in itemsUnique:
            itemsUnique.append(val)

id2label = {idx:itemsUnique for idx, itemsUnique in enumerate(itemsUnique)}
label2id = {itemsUnique:idx for idx, itemsUnique in enumerate(itemsUnique)}


binary_dataset = dataset.map(lambda examples: {"original_response": examples["response"]  if examples["original_response"] == None else examples["original_response"]})
import numpy as np
def preprocess_function(examples):
    encoding=tokenizer(examples["knowledge"], examples["original_response"], truncation=True)
    tempLables=examples["VRM"]
    lables=np.zeros(len(label2id))
    for tempLable in tempLables:
        index=label2id[tempLable]
        lables[index]=1
    encoding["labels"]=lables
    
    return encoding
encoded_dataset = binary_dataset.map(preprocess_function,remove_columns=binary_dataset['train'].column_names)

Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3539 [00:00<?, ? examples/s]

Generating test_random_split split:   0%|          | 0/1716 [00:00<?, ? examples/s]

Generating test_topic_split split:   0%|          | 0/1823 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

...

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

In [ ]:
print(dataset["train"][1])
print(encoded_dataset["train"][1])

{'dialog_idx': 0, 'response': 'Well, I know that more people started using it after some restrictions on internet use were lifited in 1995', 'original_response': 'It used to be restricted, but around 1995, the restricted were lifted and commercial use of it began', 'history': ['Can you imagine the world without internet access?', 'Yeah, but once the access to the internet was a rare thing. do you remember?', 'I do. What else can you tell me ?'], 'knowledge': 'Use by a wider audience only came in 1995 when restrictions on the use of the Internet to carry commercial traffic were lifted.', 'BEGIN': ['Hallucination', 'Entailment'], 'VRM': ['Edification']}
{'input_ids': [0, 34447, 30, 10, 6012, 2437, 129, 376, 11, 7969, 77, 5165, 15, 5, 304, 9, 5, 3742, 7, 2324, 1861, 1703, 58, 4639, 4, 2, 2, 243, 341, 7, 28, 9393, 6, 53, 198, 7969, 6, 5, 9393, 58, 4639, 8, 1861, 304, 9, 24, 880, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label2id),label2id=label2id,id2label=id2label)


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
#     warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
#     logging_steps=1000,
    evaluation_strategy='steps',
#     eval_steps=1000,
    save_total_limit=1,
#     load_best_model_at_end=True,
#     metric_for_best_model='accuracy'
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
#     compute_metrics=load_metric('accuracy')
)

# Fine-tune the model
trainer.train()

# Evaluate the model on the test set
results = trainer.evaluate(encoded_dataset['test'])
print(results)

***** Running training *****
  Num examples = 18357
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3444


Step,Training Loss,Validation Loss
500,0.198200,0.151116
1000,0.151300,0.148311
1500,0.140200,0.142411
2000,0.129000,0.135865
2500,0.120900,0.139332
3000,0.106800,0.142438


***** Running Evaluation *****
  Num examples = 3417
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3417
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3417
  Batch size = 64
Saving model checkpoint to ./resul

{'eval_loss': 0.13424207270145416, 'eval_runtime': 20.3414, 'eval_samples_per_second': 173.98, 'eval_steps_per_second': 2.753, 'epoch': 3.0}


In [ ]:
print(encoded_dataset)
encoded_dataset_test = encoded_dataset["test"]
print(encoded_dataset_test)
encoded_dataset_test_final = encoded_dataset_test.map(remove_columns="labels")
print(encoded_dataset_test_final)

predictions=trainer.predict(encoded_dataset_test_final)


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1666
    })
    valid_topic_split: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1751
    })
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3539
})


  0%|          | 0/3539 [00:00<?, ?ex/s]

***** Running Prediction *****
  Num examples = 3539
  Batch size = 64


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3539
})


In [ ]:
print(predictions)

PredictionOutput(predictions=array([[ 5.0150857, -4.270522 , -2.8931618, ..., -7.226929 , -6.9925737,
        -5.635291 ],
       [-3.163774 , -5.357179 ,  4.1297565, ..., -7.4617867, -7.4568715,
        -4.8420877],
       [-3.1685529, -5.280158 ,  4.182962 , ..., -7.478412 , -7.4858184,
        -4.8562427],
       ...,
       [ 5.1368113, -3.9223862, -2.7332063, ..., -7.4822516, -7.28288  ,
        -5.838054 ],
       [ 4.79927  , -4.0885563, -1.6859074, ..., -7.9951262, -7.8113046,
        -6.09097  ],
       [ 5.389126 , -2.80106  , -2.4682894, ..., -7.856396 , -7.690571 ,
        -6.1559873]], dtype=float32), label_ids=None, metrics={'test_runtime': 20.5019, 'test_samples_per_second': 172.618, 'test_steps_per_second': 2.731})


In [ ]:
predictionsOp=trainer.predict(encoded_dataset['test'])
print(predictionsOp)

***** Running Prediction *****
  Num examples = 3539
  Batch size = 64


PredictionOutput(predictions=array([[ 5.0150857, -4.270522 , -2.8931618, ..., -7.226929 , -6.9925737,
        -5.635291 ],
       [-3.163774 , -5.357179 ,  4.1297565, ..., -7.4617867, -7.4568715,
        -4.8420877],
       [-3.1685529, -5.280158 ,  4.182962 , ..., -7.478412 , -7.4858184,
        -4.8562427],
       ...,
       [ 5.1368113, -3.9223862, -2.7332063, ..., -7.4822516, -7.28288  ,
        -5.838054 ],
       [ 4.79927  , -4.0885563, -1.6859074, ..., -7.9951262, -7.8113046,
        -6.09097  ],
       [ 5.389126 , -2.80106  , -2.4682894, ..., -7.856396 , -7.690571 ,
        -6.1559873]], dtype=float32), label_ids=array([[1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32), metrics={'test_loss': 0.13424207270145416, 'test_runtime': 20.5967, 'test_samples_per_second': 171.823, 'test_s

In [ ]:
prediction_converted=[]
for prediction in predictionsOp.predictions:
  indiarray=[]
  for indi in prediction:
    if indi>0:
      indiarray.append(1.0)
    else:
      indiarray.append(0.0)
  prediction_converted.append(indiarray)
print(prediction_converted)

[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1

In [ ]:
print(encoded_dataset['test']["labels"])

[[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1

In [ ]:
from sklearn.metrics import f1_score
f1_score(encoded_dataset['test']["labels"], prediction_converted, average='macro')

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.46354426911532925

In [ ]:
count=0
total=0
for i in range(0,len(prediction_converted)):
  if prediction_converted[i]==encoded_dataset['test']["labels"][i]:
    count+=1
  total+=1
print("correct count",count)
print("total count",total)


correct count 2296
total count 3539


In [ ]:
print("accuracy",count/total)

accuracy 0.6487708392201187
